# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.connect("data.sqlite")
cur = conn.cursor()

In [2]:
q = """
SELECT *
FROM sqlite_master
WHERE type = 'table'
"""
pd.read_sql(q, conn)

,type,name,tbl_name,rootpage,sql
0,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
1,table,offices,offices,32,"CREATE TABLE ""offices"" (\n\t""officeCode""\tINTE..."
2,table,customers,customers,57,"CREATE TABLE ""customers"" (\n\t""customerNumber""..."
3,table,employees,employees,35,"CREATE TABLE ""employees"" (\n\t""employeeNumber""..."
4,table,orderdetails,orderdetails,2,"CREATE TABLE ""orderdetails"" (\n\t""orderNumber""..."
5,table,orders,orders,27,"CREATE TABLE ""orders"" (\n\t""orderNumber""\tINTE..."
6,table,payments,payments,28,"CREATE TABLE ""payments"" (\n\t""customerNumber""\..."
7,table,products,products,4,"CREATE TABLE ""products"" (\n\t""productCode""\tTE..."


In [3]:
# Display all table schemas
schemas_q = """
SELECT name, sql
FROM sqlite_master
"""
table_schemas = cur.execute(schemas_q).fetchall()
for schema in table_schemas:
    print(f"{schema[0]} Schema:")
    print(schema[1])

productlines Schema:
CREATE TABLE `productlines` (`productLine`, `textDescription`, `htmlDescription`, `image`)
offices Schema:
CREATE TABLE "offices" (
	"officeCode"	INTEGER,
	"city"	TEXT,
	"phone"	TEXT,
	"addressLine1"	TEXT,
	"addressLine2"	TEXT,
	"state"	TEXT,
	"country"	TEXT,
	"postalCode"	TEXT,
	"territory"	TEXT
)
customers Schema:
CREATE TABLE "customers" (
	"customerNumber"	INTEGER,
	"customerName"	TEXT,
	"contactLastName"	TEXT,
	"contactFirstName"	TEXT,
	"phone"	TEXT,
	"addressLine1"	TEXT,
	"addressLine2"	TEXT,
	"city"	TEXT,
	"state"	TEXT,
	"postalCode"	TEXT,
	"country"	TEXT,
	"salesRepEmployeeNumber"	INTEGER,
	"creditLimit"	NUMERIC
)
employees Schema:
CREATE TABLE "employees" (
	"employeeNumber"	INTEGER,
	"lastName"	TEXT,
	"firstName"	TEXT,
	"extension"	TEXT,
	"email"	TEXT,
	"officeCode"	INTEGER,
	"reportsTo"	INTEGER,
	"jobTitle"	TEXT
)
orderdetails Schema:
CREATE TABLE "orderdetails" (
	"orderNumber"	INTEGER,
	"productCode"	TEXT,
	"quantityOrdered"	INTEGER,
	"priceEach"	NUMER

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
# Your code here
q = """
SELECT
    e.firstName,
    e.lastName
FROM 
    employees AS e
    JOIN offices AS o
        USING(officeCode)
WHERE
    o.city = 'Boston'"""

pd.read_sql(q, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:
# Your code here
q = """
SELECT
    officeCode,
    o.city,
    COUNT() as num_employees
FROM
    offices AS o
    LEFT JOIN employees as e
        USING(officeCode)

GROUP BY
    officeCode;
"""
pd.read_sql(q, conn)

,officeCode,city,num_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,1


## Write 3 questions of your own and answer them

In [6]:
"""
For how many customers does each employee act as a sales representative?
"""

# Your code here
q = """
SELECT
    e.firstName AS 'First',
    e.lastName AS 'Last',
    COUNT(c.customerName) AS num_customers

FROM employees as e
    LEFT JOIN customers as c
        ON e.employeeNumber = c.salesRepEmployeeNumber

GROUP BY
    e.employeeNumber

ORDER BY
    num_customers
"""
pd.read_sql(q, conn)

,First,Last,num_customers
0,Diane,Murphy,0
1,Mary,Patterson,0
2,Jeff,Firrelli,0
3,William,Patterson,0
4,Gerard,Bondur,0
5,Anthony,Bow,0
6,Tom,King,0
7,Yoshimi,Kato,0
8,Andy,Fixter,5
9,Peter,Marsh,5


In [7]:
"""
Who are the top 5 spending customers?
"""

# Your code here
q = """
SELECT
    c.customerName AS customer_name,
    SUM(p.amount) AS total_paid

FROM customers AS c
    JOIN payments AS p
        USING(customerNumber)

GROUP BY
    customer_name
    
ORDER BY
    total_paid DESC
    
LIMIT 5;
"""

pd.read_sql(q, conn)

,customer_name,total_paid
0,Euro+ Shopping Channel,715738.98
1,Mini Gifts Distributors Ltd.,584188.24
2,"Australian Collectors, Co.",180585.07
3,Muscle Machine Inc,177913.95
4,"Dragon Souveniers, Ltd.",156251.03


In [8]:
"""
How many customers are there per office?
"""
# Your code here
q = """
SELECT 
    city,
    COUNT(customerName) AS num_customers

FROM offices AS o
    LEFT JOIN customers AS c
        USING(city)

GROUP BY
    city
    
ORDER BY
    num_customers"""

pd.read_sql(q, conn)

,city,num_customers
0,Sydney,0
1,Tokyo,0
2,London,2
3,San Francisco,2
4,Paris,3
5,Boston,4
6,NYC,5


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [9]:
# Your code here
q = """
SELECT
    e.firstName AS first_name,
    e.lastName AS last_name,
    GROUP_CONCAT(p.productName) AS product_name

FROM employees AS e
    JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING (customerNumber)
    JOIN orderdetails
        USING(orderNumber)
    JOIN products AS p
        USING(productCode)

GROUP BY 
    employeeNumber
"""
pd.read_sql(q, conn)

,first_name,last_name,product_name
0,Leslie,Jennings,"1958 Setra Bus,1940 Ford Pickup Truck,1939 Cad..."
1,Leslie,Thompson,"1917 Grand Touring Sedan,1911 Ford Town Car,19..."
2,Julie,Firrelli,"1952 Alpine Renault 1300,1940 Ford Pickup Truc..."
3,Steve,Patterson,"2001 Ferrari Enzo,1969 Corvair Monza,1969 Ford..."
4,Foon Yue,Tseng,"2001 Ferrari Enzo,1969 Corvair Monza,1969 Ford..."
5,George,Vanauf,"1969 Harley Davidson Ultimate Chopper,1996 Mot..."
6,Loui,Bondur,"1952 Alpine Renault 1300,1962 LanciaA Delta 16..."
7,Gerard,Hernandez,"1965 Aston Martin DB5,1999 Indy 500 Monte Carl..."
8,Pamela,Castillo,"1972 Alfa Romeo GTA,2001 Ferrari Enzo,1969 For..."
9,Larry,Bott,"1972 Alfa Romeo GTA,1980s Black Hawk Helicopte..."


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [10]:
# Your code here
q = """
SELECT
    e.lastName AS last_name,
    e.firstName AS first_name,
    SUM(od.quantityOrdered) AS products_sold

FROM employees AS e
    JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING (customerNumber)
    JOIN orderdetails AS od
        USING(orderNumber)

GROUP BY 
    employeeNumber

ORDER BY
    last_name
"""
pd.read_sql(q, conn)

,last_name,first_name,products_sold
0,Bondur,Loui,6186
1,Bott,Larry,8205
2,Castillo,Pamela,9290
3,Firrelli,Julie,4227
4,Fixter,Andy,6246
5,Gerard,Martin,4180
6,Hernandez,Gerard,14231
7,Jennings,Leslie,11854
8,Jones,Barry,7486
9,Marsh,Peter,6632


## Level Up 3: Display the names of employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [11]:
# Your code here
q = """
SELECT DISTINCT
    e.lastName AS last_name,
    e.firstName AS first_name,
    COUNT(p.productCode) AS different_products
FROM employees AS e
    JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING (customerNumber)
    JOIN orderdetails AS od
        USING(orderNumber)
    JOIN products as p
        USING(productCode)

GROUP BY 
    employeeNumber

HAVING
    different_products > 200

ORDER BY
    last_name
"""
pd.read_sql(q, conn)

,last_name,first_name,different_products
0,Bott,Larry,236
1,Castillo,Pamela,272
2,Hernandez,Gerard,396
3,Jennings,Leslie,331
4,Jones,Barry,220
5,Vanauf,George,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!